In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [171]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 11

5


In [172]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=575, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)',
  'GaussianNB()',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()',
  'GaussianNB(var_smoothing=1e-12)',
  'GaussianNB(var_smoothing=1e-12)'],
 ['["B_Women\'s Strawweight_rank", \'lose_streak_dif\', \'B_avg_SIG_STR_landed\', \'b

In [173]:
df = pd.read_csv("../data/ufc-master.csv")

In [174]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [175]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [176]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [177]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4208
4208
4208


In [178]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
200,Roosevelt Roberts,Brok Weaver,-360,300,27.777778,300.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.33,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0
201,Mackenzie Dern,Hannah Cifers,-420,335,23.809524,335.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
202,Katlyn Chookagian,Antonina Shevchenko,110,-130,110.000000,76.923077,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,1.00,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0
203,Daniel Rodriguez,Gabe Green,-335,275,29.850746,275.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,1.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
204,Jamahal Hill,Klidson Abreu,-117,-103,85.470085,97.087379,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4403,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1
4404,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0
4405,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4406,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Holly Holm,Irene Aldana,-125,103,80.000000,103.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Yorgan De Castro,Carlos Felipe,-265,205,37.735849,205.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Germaine de Randamie,Julianna Pena,-150,122,66.666667,122.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Kyler Phillips,Cameron Else,-480,342,20.833333,342.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Dequan Townsend,Dusko Todorovic,245,-335,245.000000,29.850746,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Alonzo Menifield,Devin Clark,-225,185,44.444444,185.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
196,Evan Dunham,Herbert Burns,185,-225,185.000000,44.444444,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
197,Tyron Woodley,Gilbert Burns,-170,150,58.823529,150.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,8.0,0.00,0.25,0.0,1.0,0.0,3.0,0.0,0.0,1
198,Blagoy Ivanov,Augusto Sakai,-105,-115,95.238095,86.956522,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,0.33,0.00,0.0,0.0,1.0,0.0,0.0,0.0,1


In [179]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


D:\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [180]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Holly Holm,Irene Aldana,-125,103,80.000000,103.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Yorgan De Castro,Carlos Felipe,-265,205,37.735849,205.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Germaine de Randamie,Julianna Pena,-150,122,66.666667,122.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Kyler Phillips,Cameron Else,-480,342,20.833333,342.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Dequan Townsend,Dusko Todorovic,245,-335,245.000000,29.850746,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Alonzo Menifield,Devin Clark,-225,185,44.444444,185.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
196,Evan Dunham,Herbert Burns,185,-225,185.000000,44.444444,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
197,Tyron Woodley,Gilbert Burns,-170,150,58.823529,150.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,8.0,0.00,0.25,0.0,1.0,0.0,3.0,0.0,0.0,1
198,Blagoy Ivanov,Augusto Sakai,-105,-115,95.238095,86.956522,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,0.33,0.00,0.0,0.0,1.0,0.0,0.0,0.0,1


In [181]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [182]:
models[1][model_num]

'GaussianNB(var_smoothing=1e-12)'

In [183]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [184]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [185]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [186]:
#1. set features
my_pos_features = ['R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [187]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_9_10_20
GaussianNB(var_smoothing=1e-12)
['B_Weight_lbs', 'B_Flyweight_rank', 'B_avg_TD_landed', 'B_Light Heavyweight_rank', 'B_Lightweight_rank', 'R_avg_TD_landed', 'R_Middleweight_rank', 'R_Height_cms', 'R_avg_SIG_STR_pct', 'B_age', 'R_longest_win_streak', 'lose_streak_dif', 'R_win_by_Decision_Majority', 'longest_win_streak_dif', 'avg_sub_att_dif', 'sig_str_dif', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'B_draw', 'R_win_by_Decision_Split', 'age_dif', 'R_odds']
2


In [188]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [189]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [21]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: 24001.954666838425
Feature: B_Reach_cms Score: 24898.174609813082
The best feature was B_Reach_cms.  It scored 24898.174609813082
Current best score is: 24898.174609813082
Feature: R_total_rounds_fought Score: 25397.989622428562
Feature: R_Lightweight_rank Score: 25898.563229882253
The best feature was R_Lightweight_rank.  It scored 25898.563229882253
Current best score is: 25898.563229882253
Feature: B_Height_cms Score: 26471.836538345855
Feature: R_Height_cms Score: 26532.71442188198
Feature: R_Weight_lbs Score: 26571.43170846892
Feature: B_Pound-for-Pound_rank Score: 26760.08755037312
The best feature was B_Pound-for-Pound_rank.  It scored 26760.08755037312
Current best score is: 26760.08755037312
Feature: R_Weight_lbs Score: 27025.131298272405
The best feature was R_Weight_lbs.  It scored 27025.131298272405
Current best score is: 27025.131298272405
Feature: B_avg_TD_pct Score: 27234.772784569683
The best feature was B_avg_TD_pct.  It scored 27234.772784569683

(3255, 29)

(3255,)

(3255, 2)

(200, 29)

...

(200, 25)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-125,103,0.696737,0.303263,0
1,-265,205,0.789357,0.210643,1
2,-150,122,0.682792,0.317208,0
3,-480,342,0.858962,0.141038,0
4,245,-335,0.607000,0.393000,1
...,...,...,...,...,...
195,-225,185,0.623310,0.376690,1
196,185,-225,0.917883,0.082117,1
197,-170,150,0.001578,0.998422,1
198,-105,-115,0.721069,0.278931,1



          Number of matches: 200
          Number of bets: 186
          Number of winning bets: 110
          Number of losing bets: 76
          Number of underdog bets: 78
          Number of underdog wins: 35
          Number of underdog losses: 43
          Number of Favorite bets: 103
          Number of favorite wins: 73
          Number of favorite losses: 30
          Number of even bets: 5
          Number of even wins: 2
          Number of even losses: 3
          Profit: 1588.6298680579941
          Profit per bet: 8.541020796010722
          Profit per match: 7.94314934028997
          
          
HI
HI
HI
HI
HI


In [190]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3264, 22)

(3264,)

(3264, 2)

(200, 22)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-125,103,0.688358,0.311642,0
1,-265,205,0.781503,0.218497,1
2,-150,122,0.741444,0.258556,0
3,-480,342,0.878509,0.121491,0
4,245,-335,0.632219,0.367781,1
...,...,...,...,...,...
195,-225,185,0.680301,0.319699,1
196,185,-225,0.999852,0.000148,1
197,-170,150,0.002975,0.997025,1
198,-105,-115,0.705360,0.294640,1



          Number of matches: 200
          Number of bets: 184
          Number of winning bets: 113
          Number of losing bets: 71
          Number of underdog bets: 79
          Number of underdog wins: 36
          Number of underdog losses: 43
          Number of Favorite bets: 99
          Number of favorite wins: 74
          Number of favorite losses: 25
          Number of even bets: 6
          Number of even wins: 3
          Number of even losses: 3
          Profit: 2347.7325834791573
          Profit per bet: 12.759416214560638
          Profit per match: 11.738662917395786
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [191]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [192]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [193]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [194]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=575, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)',
  'GaussianNB()',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()',
  'GaussianNB(var_smoothing=1e-12)',
  'GaussianNB(var_smoothing=1e-12)'],
 ['["B_Women\'s Strawweight_rank", \'lose_streak_dif\', \'B_avg_SIG_STR_landed\', \'b